# 11 - Modelo de Classificação (Prever SE vai vender)

Este notebook treina o primeiro estágio do modelo de dois estágios: **classificação**.

## Objetivo:
- Prever se um item terá vendas (`vendeu = 1`) ou não (`vendeu = 0`)
- Usar LightGBM Classifier otimizado
- Avaliar com métricas de classificação (AUC, F1-Score, Matriz de Confusão)
- Preparar para ser usado no pipeline de dois estágios

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_auc_score, f1_score, precision_score, recall_score
)
import pickle
import os
import warnings
warnings.filterwarnings('ignore')

print('🎯 Iniciando Treinamento do Modelo de Classificação')
print('🏆 Objetivo: Prever SE um item vai vender (vendeu = 1 ou 0)')

# Criar pasta submissao3 se não existir
os.makedirs('../data/submissao3', exist_ok=True)
print('📁 Pasta data/submissao3 criada/verificada')

🎯 Iniciando Treinamento do Modelo de Classificação
🏆 Objetivo: Prever SE um item vai vender (vendeu = 1 ou 0)
📁 Pasta data/submissao3 criada/verificada


## 1. Carregamento dos Dados com Features

In [2]:
# Carregar dados com features avançadas CORRIGIDAS
print('📂 Carregando dados com features CORRIGIDAS (sem target leakage)...')

# IMPORTANTE: Usar os dados CORRIGIDOS que foram salvos no Notebook 10
try:
    train_features = pd.read_parquet('../data/submissao3/train_features_CORRIGIDO.parquet')
    validation_features = pd.read_parquet('../data/submissao3/validation_features_CORRIGIDO.parquet')
    print('✅ Dados CORRIGIDOS carregados com sucesso!')
except FileNotFoundError:
    print('⚠️ Dados CORRIGIDOS não encontrados. Usando dados originais...')
    print('   Recomendação: Execute o Notebook 10 primeiro para gerar os dados corrigidos.')
    train_features = pd.read_parquet('../data/submissao3/train_features.parquet')
    validation_features = pd.read_parquet('../data/submissao3/validation_features.parquet')

print(f'🏋️ Dados de treino: {train_features.shape}')
print(f'🔍 Dados de validação: {validation_features.shape}')

# Verificar target de classificação
print(f'\n📊 Distribuição do target "vendeu" no treino:')
print(train_features['vendeu'].value_counts())
print(f'   • Proporção positiva: {train_features["vendeu"].mean()*100:.2f}%')

print(f'\n📊 Distribuição do target "vendeu" na validação:')
print(validation_features['vendeu'].value_counts())
print(f'   • Proporção positiva: {validation_features["vendeu"].mean()*100:.2f}%')

print('✅ Dados carregados e target verificado')

📂 Carregando dados com features CORRIGIDAS (sem target leakage)...
✅ Dados CORRIGIDOS carregados com sucesso!
🏋️ Dados de treino: (50126880, 54)
🔍 Dados de validação: (5221550, 54)

📊 Distribuição do target "vendeu" no treino:
vendeu
0    44583272
1     5543608
Name: count, dtype: int64
   • Proporção positiva: 11.06%

📊 Distribuição do target "vendeu" na validação:
vendeu
0    4649821
1     571729
Name: count, dtype: int64
   • Proporção positiva: 10.95%
✅ Dados carregados e target verificado


## 2. Preparação dos Dados para Classificação

In [3]:
# Definir features para o modelo de classificação SEM TARGET LEAKAGE
print('🔧 Preparando dados para classificação SEM TARGET LEAKAGE...')

# CRÍTICO: Excluir features que causam vazamento de informação
features_excluir = [
    # IDs e data
    'semana', 'pdv_id', 'produto_id',  
    # Targets
    'quantidade', 'faturamento', 'vendeu',       
    # IDs auxiliares
    'distributor_id',                  
    # Categóricas brutas (usamos as hash)
    'categoria', 'zipcode', 'tipo_loja',  
    
    # 🚨 CRÍTICO: Features que causam TARGET LEAKAGE (usam informação da semana atual)
    'preco_unitario_atual',                     # Usa quantidade atual  
    'preco_medio_semanal_sku_atual',           # Usa preço atual
    'media_vendas_categoria_pdv_atual',        # Usa quantidade atual
    'share_vendas_sku_categoria_atual',        # Usa quantidade atual
    
    # REMOVIDAS do modelo original que causavam leakage
    'preco_unitario',                          # Calculado com quantidade atual
    'preco_medio_semanal_sku',                # Usa preço atual  
    'preco_medio_semanal_categoria_pdv',      # Usa preço atual
    'media_vendas_categoria_no_pdv',          # Usa quantidade atual
    'media_vendas_sku_no_zipcode',            # Usa quantidade atual  
    'share_vendas_sku_na_categoria'           # Usa quantidade atual
]

# Selecionar features SEGURAS (sem leakage)
features_disponiveis = list(train_features.columns)
features_modelo_seguros = [col for col in features_disponiveis if col not in features_excluir]

print(f'📊 Features disponíveis: {len(features_disponiveis)}')
print(f'📊 Features SEGURAS para classificação: {len(features_modelo_seguros)}')
print(f'📊 Features excluídas (com leakage): {len(features_excluir)}')

# Mostrar features SEGURAS que serão usadas
print('\n✅ FEATURES SEGURAS (sem target leakage):')
for feat in features_modelo_seguros:
    print(f'   • {feat}')

# Mostrar features REMOVIDAS (com leakage)  
print('\n🚨 FEATURES REMOVIDAS (causavam leakage):')
features_com_leakage = [f for f in features_excluir if 'preco_unitario' in f or 'media_vendas' in f or 'share_vendas' in f or 'preco_medio' in f]
for feat in features_com_leakage:
    if feat in features_disponiveis:
        print(f'   ❌ {feat}')

# Preparar datasets COM FEATURES SEGURAS
X_train = train_features[features_modelo_seguros].copy()
y_train = train_features['vendeu'].copy()

X_val = validation_features[features_modelo_seguros].copy() 
y_val = validation_features['vendeu'].copy()

print(f'\n📊 Datasets preparados (SEM LEAKAGE):')
print(f'   🏋️ X_train: {X_train.shape}, y_train: {y_train.shape}')
print(f'   🔍 X_val: {X_val.shape}, y_val: {y_val.shape}')

# Verificar se há NAs
nas_train = X_train.isnull().sum().sum()
nas_val = X_val.isnull().sum().sum()
print(f'   🧹 NAs no treino: {nas_train}, NAs na validação: {nas_val}')

if nas_train > 0 or nas_val > 0:
    print('   ⚠️ Preenchendo NAs com 0...')
    X_train = X_train.fillna(0)
    X_val = X_val.fillna(0)

# Verificar se ainda temos features suficientes
if len(features_modelo_seguros) < 10:
    print('⚠️ AVISO: Poucas features disponíveis! Pode indicar problema na engenharia de features.')
else:
    print(f'✅ {len(features_modelo_seguros)} features seguras - suficientes para treinamento!')

print('✅ Dados preparados para classificação SEM TARGET LEAKAGE')

🔧 Preparando dados para classificação SEM TARGET LEAKAGE...
📊 Features disponíveis: 54
📊 Features SEGURAS para classificação: 41
📊 Features excluídas (com leakage): 20

✅ FEATURES SEGURAS (sem target leakage):
   • preco_lag_1
   • preco_lag_2
   • variacao_preco_sku_semanal
   • quantidade_lag_1
   • quantidade_lag_2
   • quantidade_lag_3
   • quantidade_lag_4
   • quantidade_media_4w
   • quantidade_std_4w
   • quantidade_max_4w
   • quantidade_ewma_4w
   • quantidade_ewma_8w
   • preco_ewma_4w
   • semana_do_ano
   • eh_primeira_semana_mes
   • eh_dezembro
   • eh_janeiro
   • eh_pos_festas
   • semana_ano_sin
   • semana_ano_cos
   • preco_relativo_categoria
   • preco_relativo_pdv
   • preco_volatilidade
   • media_vendas_categoria_pdv_lag_1
   • share_vendas_sku_categoria_lag_1
   • momentum_ratio
   • momentum_ratio_ewma
   • aceleracao
   • dia_do_mes
   • semana_do_mes
   • eh_inicio_mes
   • eh_fim_mes
   • mes
   • mes_sin
   • mes_cos
   • pdv_hash
   • produto_hash
   • ca

## 3. Treinamento do Modelo LightGBM Classifier

In [4]:
# Configurar e treinar LightGBM Classifier
print('🚀 Treinando LightGBM Classifier...')

# Parâmetros otimizados para classificação
lgbm_params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 100,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 100,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'verbose': -1,
    'random_state': 42,
    'n_jobs': -1
}

# Criar modelo
lgbm_classifier = lgb.LGBMClassifier(**lgbm_params)

# Treinar modelo com callbacks para early stopping
print('   📚 Iniciando treinamento...')
lgbm_classifier.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric=['binary_logloss', 'auc'],
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
)

print(f'✅ Modelo treinado! Melhor iteração: {lgbm_classifier.best_iteration_}')
print(f'📊 Score de validação: {lgbm_classifier.best_score_}')

🚀 Treinando LightGBM Classifier...
   📚 Iniciando treinamento...
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.84728	valid_0's binary_logloss: 0.267367
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.84728	valid_0's binary_logloss: 0.267367
✅ Modelo treinado! Melhor iteração: 100
📊 Score de validação: defaultdict(<class 'collections.OrderedDict'>, {'valid_0': OrderedDict([('auc', np.float64(0.8472803725977874)), ('binary_logloss', np.float64(0.2673666610763523))])})


## 4. Avaliação do Modelo

In [5]:
# Fazer previsões
print('🔍 Avaliando modelo de classificação...')

# Previsões de probabilidade
y_pred_proba_train = lgbm_classifier.predict_proba(X_train)[:, 1]
y_pred_proba_val = lgbm_classifier.predict_proba(X_val)[:, 1]

# Previsões binárias (threshold = 0.5)
y_pred_train = lgbm_classifier.predict(X_train)
y_pred_val = lgbm_classifier.predict(X_val)

# Métricas de treino
print('\n📊 MÉTRICAS DE TREINO:')
print(f'   🎯 AUC: {roc_auc_score(y_train, y_pred_proba_train):.4f}')
print(f'   🎯 F1-Score: {f1_score(y_train, y_pred_train):.4f}')
print(f'   🎯 Precision: {precision_score(y_train, y_pred_train):.4f}')
print(f'   🎯 Recall: {recall_score(y_train, y_pred_train):.4f}')

# Métricas de validação
print('\n📊 MÉTRICAS DE VALIDAÇÃO:')
auc_val = roc_auc_score(y_val, y_pred_proba_val)
f1_val = f1_score(y_val, y_pred_val)
precision_val = precision_score(y_val, y_pred_val)
recall_val = recall_score(y_val, y_pred_val)

print(f'   🎯 AUC: {auc_val:.4f}')
print(f'   🎯 F1-Score: {f1_val:.4f}')
print(f'   🎯 Precision: {precision_val:.4f}')
print(f'   🎯 Recall: {recall_val:.4f}')

# Matriz de confusão
print('\n📊 MATRIZ DE CONFUSÃO (Validação):')
cm = confusion_matrix(y_val, y_pred_val)
print(f'                 Predito')
print(f'               0       1')
print(f'Real    0   {cm[0,0]:7,} {cm[0,1]:7,}')
print(f'        1   {cm[1,0]:7,} {cm[1,1]:7,}')

# Análise de diferentes thresholds
print('\n📊 ANÁLISE DE THRESHOLDS:')
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]
for threshold in thresholds:
    y_pred_thresh = (y_pred_proba_val >= threshold).astype(int)
    f1_thresh = f1_score(y_val, y_pred_thresh)
    precision_thresh = precision_score(y_val, y_pred_thresh)
    recall_thresh = recall_score(y_val, y_pred_thresh)
    print(f'   Threshold {threshold:.1f}: F1={f1_thresh:.4f}, Prec={precision_thresh:.4f}, Rec={recall_thresh:.4f}')

print('✅ Avaliação do modelo concluída')

🔍 Avaliando modelo de classificação...

📊 MÉTRICAS DE TREINO:
   🎯 AUC: 0.8784
   🎯 F1-Score: 0.3917
   🎯 Precision: 0.7043
   🎯 Recall: 0.2713

📊 MÉTRICAS DE VALIDAÇÃO:
   🎯 AUC: 0.8473
   🎯 F1-Score: 0.1460
   🎯 Precision: 0.7078
   🎯 Recall: 0.0814

📊 MATRIZ DE CONFUSÃO (Validação):
                 Predito
               0       1
Real    0   4,630,608  19,213
        1   525,188  46,541

📊 ANÁLISE DE THRESHOLDS:
   Threshold 0.3: F1=0.3520, Prec=0.5647, Rec=0.2557
   Threshold 0.4: F1=0.2366, Prec=0.6448, Rec=0.1449
   Threshold 0.5: F1=0.1460, Prec=0.7078, Rec=0.0814
   Threshold 0.6: F1=0.0708, Prec=0.7642, Rec=0.0371
   Threshold 0.7: F1=0.0089, Prec=0.8442, Rec=0.0045
✅ Avaliação do modelo concluída


## 5. Análise de Importância das Features

In [6]:
# Analisar importância das features
print('📊 Analisando importância das features...')

# Obter importâncias
feature_importance = lgbm_classifier.feature_importances_
feature_names = X_train.columns

# Criar DataFrame de importâncias
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

# Top 20 features mais importantes
print('\n🏆 TOP 20 FEATURES MAIS IMPORTANTES:')
for i, (_, row) in enumerate(importance_df.head(20).iterrows()):
    print(f'   {i+1:2d}. {row["feature"]:30s} - {row["importance"]:6.0f}')

# Análise por categoria de feature
print('\n📊 IMPORTÂNCIA POR CATEGORIA:')
categorias_feature = {
    'Lag': [f for f in feature_names if 'lag_' in f],
    'Rolling': [f for f in feature_names if any(x in f for x in ['media_4w', 'std_4w', 'max_4w'])],
    'Preço': [f for f in feature_names if 'preco' in f],
    'Calendário': [f for f in feature_names if any(x in f for x in ['mes', 'dia', 'inicio', 'fim'])],
    'Tendência': [f for f in feature_names if any(x in f for x in ['momentum', 'aceleracao'])],
    'Hierarquia': [f for f in feature_names if any(x in f for x in ['media_vendas', 'share'])],
    'Hash': [f for f in feature_names if 'hash' in f]
}

for categoria, features in categorias_feature.items():
    if features:
        importancia_categoria = importance_df[importance_df['feature'].isin(features)]['importance'].sum()
        print(f'   {categoria:12s}: {importancia_categoria:8.0f} ({len(features)} features)')

print('✅ Análise de importância concluída')

📊 Analisando importância das features...

🏆 TOP 20 FEATURES MAIS IMPORTANTES:
    1. quantidade_ewma_8w             -   1271
    2. preco_ewma_4w                  -    972
    3. quantidade_ewma_4w             -    922
    4. semana_do_ano                  -    890
    5. preco_lag_1                    -    867
    6. categoria_hash                 -    622
    7. preco_relativo_categoria       -    622
    8. produto_hash                   -    357
    9. media_vendas_categoria_pdv_lag_1 -    354
   10. preco_lag_2                    -    326
   11. preco_relativo_pdv             -    307
   12. momentum_ratio_ewma            -    294
   13. quantidade_media_4w            -    281
   14. categoria_zipcode_hash         -    240
   15. preco_volatilidade             -    212
   16. quantidade_std_4w              -    185
   17. semana_ano_sin                 -    183
   18. mes_sin                        -    117
   19. quantidade_max_4w              -    114
   20. aceleracao          

## 6. Salvamento do Modelo

In [7]:
# Salvar modelo treinado SEM TARGET LEAKAGE
print('💾 Salvando modelo de classificação SEM LEAKAGE...')

# Salvar modelo
with open('../data/submissao3/lgbm_classifier_CORRIGIDO.pkl', 'wb') as f:
    pickle.dump(lgbm_classifier, f)

# CRÍTICO: Salvar lista de features SEGURAS (sem leakage)
with open('../data/submissao3/classification_features_CORRIGIDO.pkl', 'wb') as f:
    pickle.dump(features_modelo_seguros, f)

# Salvar importâncias
importance_df.to_csv('../data/submissao3/classification_feature_importance_CORRIGIDO.csv', index=False)

# Salvar metadados do modelo CORRIGIDO
metadados_classifier = {
    'data_criacao': pd.Timestamp.now(),
    'modelo': 'LightGBM Classifier SEM TARGET LEAKAGE',
    'objetivo': 'Classificação: prever se vai vender (vendeu = 1 ou 0)',
    'parametros': lgbm_params,
    'melhor_iteracao': lgbm_classifier.best_iteration_,
    'melhor_score': lgbm_classifier.best_score_,
    'total_features': len(features_modelo_seguros),
    'features_usadas': features_modelo_seguros,
    'features_removidas_por_leakage': features_com_leakage,
    'correcao_target_leakage': {
        'problema_original': 'Rolling features incluíam valor da semana atual (target leakage)',
        'solucao_aplicada': 'Rolling/EWMA baseados em quantidade_lag_1, não quantidade atual',
        'dados_corrigidos': 'Usado train_features_CORRIGIDO.parquet',
        'features_removidas': len(features_com_leakage),
        'features_seguras_restantes': len(features_modelo_seguros)
    },
    'metricas_validacao': {
        'auc': auc_val,
        'f1_score': f1_val,
        'precision': precision_val,
        'recall': recall_val
    },
    'shape_treino': X_train.shape,
    'shape_validacao': X_val.shape,
    'distribuicao_target_treino': y_train.value_counts().to_dict(),
    'distribuicao_target_validacao': y_val.value_counts().to_dict(),
    'observacoes': [
        'VERSÃO CORRIGIDA: Rolling features baseadas em dados defasados',
        'Notebook 10 foi corrigido para calcular features sobre quantidade_lag_1',
        'Features que usavam informação atual foram removidas',
        'Dados carregados de train_features_CORRIGIDO.parquet'
    ]
}

with open('../data/submissao3/lgbm_classifier_CORRIGIDO_metadata.pkl', 'wb') as f:
    pickle.dump(metadados_classifier, f)

print('✅ Arquivos CORRIGIDOS salvos:')
print('   • data/submissao3/lgbm_classifier_CORRIGIDO.pkl')
print('   • data/submissao3/classification_features_CORRIGIDO.pkl')
print('   • data/submissao3/classification_feature_importance_CORRIGIDO.csv')
print('   • data/submissao3/lgbm_classifier_CORRIGIDO_metadata.pkl')

print('\\n🎉 MODELO DE CLASSIFICAÇÃO CORRIGIDO TREINADO!')
print('=' * 70)
print('🎯 Resumo da Correção:')
print(f'   ❌ Problema original: Rolling features incluíam semana atual')
print(f'   ✅ Correção aplicada: Rolling baseado em quantidade_lag_1')
print(f'   📊 Features seguras usadas: {len(features_modelo_seguros)}')
print(f'   📊 AUC: {auc_val:.4f}')
print(f'   📊 F1-Score: {f1_val:.4f}')

# Diagnóstico final
if auc_val > 0.999:
    print('\\n⚠️ AINDA HAY TARGET LEAKAGE DETECTADO!')
    print('   🔍 AUC muito alto (>0.999) indica vazamento de informação')
    print('   💡 Recomendações:')
    print('   1. Verificar se os dados CORRIGIDOS foram usados')
    print('   2. Re-executar Notebook 10 com as correções aplicadas')
    print('   3. Verificar se rolling features estão baseadas em lags')
elif auc_val > 0.95:
    print('\\n✅ TARGET LEAKAGE CORRIGIDO COM SUCESSO!')
    print('   🎯 AUC em faixa realista (0.95-0.99)')
    print('   📊 Métricas indicam modelo funcional')
else:
    print('\\n🤔 AUC MUITO BAIXO - POSSÍVEL PROBLEMA')
    print('   ⚠️ AUC < 0.95 pode indicar features insuficientes')
    print('   🔍 Verifique se features importantes não foram removidas')

print('\\n💡 Próximo passo:')
print('   🔄 Treinar modelo de regressão (Notebook 12)')
print('   📊 Pode usar features atuais no regressor (quando vendeu=1)')

print('\\n🚀 Primeiro estágio do modelo de dois estágios!')

💾 Salvando modelo de classificação SEM LEAKAGE...
✅ Arquivos CORRIGIDOS salvos:
   • data/submissao3/lgbm_classifier_CORRIGIDO.pkl
   • data/submissao3/classification_features_CORRIGIDO.pkl
   • data/submissao3/classification_feature_importance_CORRIGIDO.csv
   • data/submissao3/lgbm_classifier_CORRIGIDO_metadata.pkl
\n🎉 MODELO DE CLASSIFICAÇÃO CORRIGIDO TREINADO!
🎯 Resumo da Correção:
   ❌ Problema original: Rolling features incluíam semana atual
   ✅ Correção aplicada: Rolling baseado em quantidade_lag_1
   📊 Features seguras usadas: 41
   📊 AUC: 0.8473
   📊 F1-Score: 0.1460
\n🤔 AUC MUITO BAIXO - POSSÍVEL PROBLEMA
   ⚠️ AUC < 0.95 pode indicar features insuficientes
   🔍 Verifique se features importantes não foram removidas
\n💡 Próximo passo:
   🔄 Treinar modelo de regressão (Notebook 12)
   📊 Pode usar features atuais no regressor (quando vendeu=1)
\n🚀 Primeiro estágio do modelo de dois estágios!
